# C**hapter 12: Advanced TensorFlow Implementation Guide**

## 1. Introduction to TensorFlow Customization

TensorFlow provides multiple levels of abstraction for building and training models. This chapter focuses on going beyond the high-level Keras API to create custom components and training loops.

## 2. Custom Loss Functions

### 2.1 Creating Custom Losses
TensorFlow allows you to define your own loss functions when standard ones don't meet your needs.

**Example: Huber Loss Implementation**

The Huber loss is less sensitive to outliers than MSE:
\[
L_{\delta}(y, \hat{y}) = \begin{cases}
\frac{1}{2}(y - \hat{y})^2 & \text{for } |y - \hat{y}| \leq \delta \\
\delta(|y - \hat{y}| - \frac{1}{2}\delta) & \text{otherwise}
\end{cases}
\]

In [1]:
# Mengimpor pustaka Python dan TensorFlow yang diperlukan
import tensorflow as tf

def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < delta
    squared_loss = tf.square(error) / 2
    linear_loss = delta * (tf.abs(error) - delta/2)
    return tf.where(is_small_error, squared_loss, linear_loss)

# Test the function
y_true = tf.constant([0., 1., 2.])
y_pred = tf.constant([0.1, 1.2, 1.9])
print("Huber loss:", huber_loss(y_true, y_pred))

Huber loss: tf.Tensor([0.005      0.02000001 0.005     ], shape=(3,), dtype=float32)


### 2.2 Using Custom Losses in Keras Models

You can use custom loss functions just like built-in ones:

In [2]:
# Mengimpor pustaka Python dan TensorFlow yang diperlukan
from tensorflow import keras
# Mengimpor pustaka Python dan TensorFlow yang diperlukan
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(10, activation='relu', input_shape=(8,)),
    layers.Dense(1)
])

# Menyusun model dengan fungsi loss dan optimizer
model.compile(optimizer='adam', loss=huber_loss)
# Menyusun model dengan fungsi loss dan optimizer
print("Model compiled with custom Huber loss")

Model compiled with custom Huber loss


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 3. Custom Layers

### 3.1 Implementing a Custom Layer

Create layers by subclassing `tf.keras.layers.Layer`:

In [3]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_normal',
            trainable=True)
        self.bias = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True)
        super().build(input_shape)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units, 'activation': tf.keras.activations.serialize(self.activation)}

# Test the custom layer
custom_layer = MyDenseLayer(32, activation='relu')
print("Output shape:", custom_layer(tf.zeros([1, 10])).shape)

Output shape: (1, 32)


## 4. Custom Training Loops

### 4.1 Basic Training Loop Structure

When you need more control than `model.fit()`, implement your own loop:

In [4]:
# Create a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MeanSquaredError()

# Generate dummy data
# Mengimpor pustaka Python dan TensorFlow yang diperlukan
import numpy as np
x_train = np.random.random((1000, 8))
y_train = np.random.random((1000, 1))

batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

epochs = 3
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch + 1}")

    for step, (x_batch, y_batch) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 10 == 0:
            print(f"Training loss at step {step}: {float(loss_value):.4f}")


Start of epoch 1
Training loss at step 0: 0.1963
Training loss at step 10: 0.2318
Training loss at step 20: 0.1428
Training loss at step 30: 0.1639

Start of epoch 2
Training loss at step 0: 0.1811
Training loss at step 10: 0.1787
Training loss at step 20: 0.1639
Training loss at step 30: 0.2190

Start of epoch 3
Training loss at step 0: 0.0970
Training loss at step 10: 0.1524
Training loss at step 20: 0.1310
Training loss at step 30: 0.1339


## 5. Using tf.function for Performance

### 5.1 Converting Python Functions to TensorFlow Graphs

`@tf.function` decorator converts Python functions to optimized TensorFlow graphs:

In [5]:
@tf.function
def my_function(x):
    return tf.reduce_sum(x ** 2)

x = tf.constant([1.0, 2.0, 3.0])
print("Function output:", my_function(x))

Function output: tf.Tensor(14.0, shape=(), dtype=float32)


## 6. Exercises

1. Implement a custom layer that performs layer normalization
2. Create a custom metric that tracks precision for a binary classifier
3. Modify the training loop to include learning rate scheduling
4. Benchmark a model with and without `@tf.function` to see the performance difference

## 7. Key Takeaways

- TensorFlow provides flexible APIs for customizing every aspect of model building and training
- Custom components should subclass appropriate Keras base classes for compatibility
- **Training loop kustom** memberikan kontrol penuh atas proses pelatihan, berguna untuk eksperimen lanjutan. offer maximum control but require more code
- `@tf.function` can significantly improve performance by converting Python to graph execution